In [5]:
def calculate_output_dimensions(input_dim, layer_config):
    # Initialize input dimensions
    H, W, C = input_dim
    
    # Store output dimensions and parameter counts for each layer
    results = []
    results.append(("Input", f"{H}×{W}×{C}", 0))
    
    for layer in layer_config:
        if layer.startswith("CONV"):
            # Parse convolution layer parameters
            parts = layer.split(" ")
            kernel_size = int(parts[1].split("-")[0])
            filters = int(parts[1].split("-")[1])
            padding = 1
            stride = 1
            
            # Calculate output dimensions
            H_out = (H + 2*padding - kernel_size)//stride + 1
            W_out = (W + 2*padding - kernel_size)//stride + 1
            C_out = filters
            
            # Calculate parameter count (kernel_size^2 * C_in * C_out + C_out)
            params = (kernel_size * kernel_size * C + 1) * filters
            
            results.append((layer, f"{H_out}×{W_out}×{C_out}", params))
            
            # Update dimensions
            H, W, C = H_out, W_out, C_out
            
        elif layer == "Leaky ReLU":
            # Leaky ReLU does not change dimensions and has no parameters
            results.append((layer, f"{H}×{W}×{C}", 0))
            
        elif layer.startswith("POOL"):
            # Parse pooling layer parameters
            pool_size = int(layer.split("-")[1])
            stride = pool_size
            
            # Calculate output dimensions
            H_out = (H - pool_size)//stride + 1
            W_out = (W - pool_size)//stride + 1
            
            results.append((layer, f"{H_out}×{W_out}×{C}", 0))
            
            # Update dimensions
            H, W = H_out, W_out
            
        elif layer == "FLATTEN":
            # Calculate dimensions after flattening
            flattened_size = H * W * C
            results.append((layer, f"1×1×{flattened_size}", 0))
            
            # Update dimensions
            H, W, C = 1, 1, flattened_size
            
        elif layer.startswith("FC"):
            # Parse fully connected layer parameters
            output_size = int(layer.split("-")[1])
            
            # Calculate parameter count (input_size * output_size + output_size)
            params = (H * W * C + 1) * output_size
            
            results.append((layer, f"1×1×{output_size}", params))
            
            # Update dimensions
            C = output_size
    
    return results

# Layer configuration
layer_config = [
    "CONV 3-16",
    "Leaky ReLU",
    "POOL-2",
    "CONV 3-32",
    "Leaky ReLU", 
    "POOL-2",
    "FLATTEN",
    "FC-10"
]

# Calculate and print results
input_dim = (28, 28, 3)
results = calculate_output_dimensions(input_dim, layer_config)

# Print results table
print("| Layer      | Dimensions | Parameter Count (weights and biases) |")
print("| ---------- | ----------- | ------------------------------------ |")
for layer, dim, params in results:
    print(f"| {layer:<10} | {dim:<8} | {params:<25} |")

| Layer      | Dimensions | Parameter Count (weights and biases) |
| ---------- | ----------- | ------------------------------------ |
| Input      | 28×28×3  | 0                         |
| CONV 3-16  | 28×28×16 | 448                       |
| Leaky ReLU | 28×28×16 | 0                         |
| POOL-2     | 14×14×16 | 0                         |
| CONV 3-32  | 14×14×32 | 4640                      |
| Leaky ReLU | 14×14×32 | 0                         |
| POOL-2     | 7×7×32   | 0                         |
| FLATTEN    | 1×1×1568 | 0                         |
| FC-10      | 1×1×10   | 15690                     |
